In [ ]:
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Reshape, Conv2DTranspose

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
from glob import glob
from tensorflow.keras.preprocessing import image

In [ ]:
from tensorflow.keras.layers import Dense,Input,Flatten,Dropout,SimpleRNN,LSTM,GlobalMaxPooling1D,Embedding,Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
! kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 144MB/s]


In [ ]:
!unzip -qq twitter-entity-sentiment-analysis.zip

In [ ]:
train_ds = pd.read_csv('twitter_training.csv',header=None,names=['id','topic','target','text'])
test_ds = pd.read_csv('twitter_validation.csv',header=None,names=['id','topic','target','text'])

In [ ]:
train_ds.head()

,id,topic,target,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
train_ds['target'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [ ]:
train_ds['encoded_target']=train_ds['target'].map({'Positive':1,'Negative':0,'Neutral':2,'Irrelevant':3})
test_ds['encoded_target']=test_ds['target'].map({'Positive':1,'Negative':0,'Neutral':2,'Irrelevant':3})

In [ ]:
train_ds.head()

,id,topic,target,text,encoded_target
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,1
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,1
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,1
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,1
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,1


In [ ]:
x_train,y_train = train_ds['text'],train_ds['encoded_target']
x_test,y_test = test_ds['text'],test_ds['encoded_target']

In [ ]:
x_train=x_train.astype(str)
x_test = x_test.astype(str)

In [ ]:
MAX_VOCAB_SIZE=20000

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE) # utile per evitare le parole rare o mispelling, et similia
tokenizer.fit_on_texts(x_train) # basta solo questa fit perché si suppone ci siano le stesse parole che nel test
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq=tokenizer.texts_to_sequences(x_test)

In [ ]:
tokenizer.word_index

{'the': 1,
 'i': 2,
 'to': 3,
 'and': 4,
 'a': 5,
 'of': 6,
 'is': 7,
 'in': 8,
 'for': 9,
 'this': 10,
 'it': 11,
 'you': 12,
 'on': 13,
 'my': 14,
 'that': 15,
 'com': 16,
 'with': 17,
 'game': 18,
 'so': 19,
 'be': 20,
 'me': 21,
 'have': 22,
 'just': 23,
 'but': 24,
 'not': 25,
 'are': 26,
 'all': 27,
 'at': 28,
 'was': 29,
 'like': 30,
 'out': 31,
 'from': 32,
 '2': 33,
 'your': 34,
 'pic': 35,
 'twitter': 36,
 'now': 37,
 'get': 38,
 'we': 39,
 'as': 40,
 'they': 41,
 'has': 42,
 'if': 43,
 'one': 44,
 'do': 45,
 'good': 46,
 't': 47,
 'about': 48,
 'can': 49,
 'play': 50,
 'no': 51,
 'will': 52,
 'an': 53,
 'new': 54,
 'really': 55,
 'love': 56,
 'when': 57,
 'up': 58,
 "i'm": 59,
 'unk': 60,
 'what': 61,
 'more': 62,
 'time': 63,
 'by': 64,
 'johnson': 65,
 'how': 66,
 'people': 67,
 'some': 68,
 'or': 69,
 'why': 70,
 '3': 71,
 'see': 72,
 'shit': 73,
 "it's": 74,
 'co': 75,
 'been': 76,
 'best': 77,
 'still': 78,
 'facebook': 79,
 '’': 80,
 'https': 81,
 'got': 82,
 'games': 

In [ ]:
T = 0
for el in x_train_seq:
  
  if len(el)>T:
    T=len(el)
for el in x_test_seq:
  if len(el)>T:
    T=len(el)
T

166

In [ ]:
x_train_data = pad_sequences(x_train_seq,maxlen=T)
x_test_data=pad_sequences(x_test_seq,maxlen=T)

In [ ]:
V=len(tokenizer.word_index)
V # unique token

33784

In [ ]:
K=len(y_train.unique())
K

4

'im getting on borderlands and i will murder you all ,'

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, SpatialDropout1D, Embedding, LSTM
model = Sequential()

In [ ]:
n_most_words = V+1
n_dim = 50

In [ ]:
D=50

M=50



i = Input(shape=(T,))
x=Embedding(V+1,D)(i) # mettiamo +1 poiché partiamo da 1, non da 0, quindi ce ne serve uno in più, in quanto 
                    # gli array su python iniziano da 0
x=LSTM(M)(x)


x = Dropout(0.2)(x)

x = Dense(25,activation='relu')(x)

x = Dropout(0.2)(x)

x=Dense(K,activation='softmax')(x)

model = Model(i,x)

model.compile(
    optimizer = tf.optimizers.Adam(lr=0.001),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

r = model.fit(x_train_data,y_train,
              validation_data=(x_test_data,y_test),
              epochs=10
              )

Epoch 1/10
2334/2334 [==============================] - 252s 107ms/step - loss: 0.8117 - accuracy: 0.6814 - val_loss: 0.2714 - val_accuracy: 0.9210
Epoch 2/10
2334/2334 [==============================] - 245s 105ms/step - loss: 0.3794 - accuracy: 0.8626 - val_loss: 0.2115 - val_accuracy: 0.9440
Epoch 3/10
2334/2334 [==============================] - 245s 105ms/step - loss: 0.2727 - accuracy: 0.8976 - val_loss: 0.2212 - val_accuracy: 0.9430
Epoch 4/10
2334/2334 [==============================] - 245s 105ms/step - loss: 0.2225 - accuracy: 0.9148 - val_loss: 0.2305 - val_accuracy: 0.9540
Epoch 5/10
2334/2334 [==============================] - 246s 105ms/step - loss: 0.1886 - accuracy: 0.9269 - val_loss: 0.2087 - val_accuracy: 0.9490
Epoch 6/10
2334/2334 [==============================] - 247s 106ms/step - loss: 0.1665 - accuracy: 0.9335 - val_loss: 0.2332 - val_accuracy: 0.9480
Epoch 7/10
2334/2334 [==============================] - 247s 106ms/step - loss: 0.1494 - accuracy: 0.9397 - val_